In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math

In [ ]:
def EV(str1,str2,checkerboardsize,k):
    img_1=cv2.cvtColor(cv2.imread(str1),cv2.COLOR_BGR2GRAY).astype(np.float32)
    zd=np.genfromtxt(str2, delimiter=',')
    zx=zd[k][1]/1000
    h_m=harris_corner(img_1)

    l,m=0,0
    y=np.zeros((8,11,2))
    r,c=h_m.shape
    for i in range(0,1024):
        for j in range(0,1024):
          if(h_m[i][j]==255):
              y[l,m,0]=i
              m=m+1
              if(m==11):
                l=l+1
                m=0
    l=0
    m=0
    for j in range(0,1024):
        for i in range(0,1024):
          if(h_m[i][j]==255):
              y[l,m,1]=j
              l=l+1
              if(l==8):
                m=m+1
                l=0
    l=0
    m=0
    X=np.array([[0,0,zx/1000,1]])
    z=np.zeros((1,4))
    l1=np.hstack((z,-X,y[0,0,1]*X))
    l2=np.hstack((X,z,-y[0,0,0]*X))
    a= np.vstack((l1,l2))
    for i in range(8):
      for j in range(11):
        if(i!=0 or j!=0):
          X=np.array([[i*0.025,j*0.025,zx/1000,1]])
          z=np.zeros((1,4))
          l1=np.hstack((z,-X,y[i,j,1]*X))
          l2=np.hstack((X,z,-y[i,j,0]*X))
          a= np.vstack((a,l2))
          a= np.vstack((a,l1))
    rank = np.linalg.matrix_rank(a)
    U, S, VT = np.linalg.svd(a,full_matrices=False)
    eigenvector_min=VT[rank,:]

    return eigenvector_min




In [ ]:
def harris_corner(img_1):

  def sobel_filter(image):

    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])

    sobel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]])
    grad_x = cv2.filter2D(image, -1, sobel_x)
    grad_y = cv2.filter2D(image, -1, sobel_y)

    return grad_x, grad_y

  def harris_corner_response(image, k=0.04):
    grad_x, grad_y = sobel_filter(image)
    Ixx = grad_x ** 2
    Iyy = grad_y ** 2
    Ixy = grad_x * grad_y

    window_size = 3
    Ixx_sum = cv2.GaussianBlur(Ixx, (window_size, window_size), 0)
    Iyy_sum = cv2.GaussianBlur(Iyy, (window_size, window_size), 0)
    Ixy_sum = cv2.GaussianBlur(Ixy, (window_size, window_size), 0)
    det_M = Ixx_sum * Iyy_sum - Ixy_sum ** 2
    trace_M = Ixx_sum + Iyy_sum
    R = det_M - k * (trace_M ** 2)
    return R
  def non_maximum_suppression(R, threshold=0.01):
    R[R < threshold * np.max(R)] = 0
    corners = np.zeros_like(R)
    for i in range(1, R.shape[0] - 1):
        for j in range(1, R.shape[1] - 1):
            if R[i, j] > 0:
                if (R[i, j] > R[i - 1, j - 1] and
                    R[i, j] > R[i - 1, j] and
                    R[i, j] > R[i - 1, j + 1] and
                    R[i, j] > R[i, j - 1] and
                    R[i, j] > R[i, j + 1] and
                    R[i, j] > R[i + 1, j - 1] and
                    R[i, j] > R[i + 1, j] and
                    R[i, j] > R[i + 1, j + 1]):
                    corners[i, j] = 255

    return corners
  image =img_1
  R = harris_corner_response(image)
  corners = non_maximum_suppression(R)
  return corners


In [ ]:
def p_matrix(x):
  P=x.reshape((3,4))
  P=P/P[-1,-1]
  P=P[:,:3]
  m1,m2,m3=P[0,:],P[1],P[2]
  r3=m3/np.linalg.norm(m3)
  e2=m2-(np.dot(m2.T,r3.T)*r3)
  r2=e2/np.linalg.norm(e2)
  e1=m1-(np.dot(m1.T,r3.T)*r3)-(np.dot(m1.T,r2.T)*r2)
  r1=e1/np.linalg.norm(e1)
  K=np.zeros((3,3))
  K[0,0]=np.dot(e1.T,r1.T)
  K[0,1]=np.dot(m1.T,r2.T)
  K[0,2]=np.dot(m1.T,r3.T)
  K[1,1]=np.dot(e2.T,r2.T)
  K[1,2]=np.dot(m2.T,r2.T)
  K[2,2]=np.dot(m3.T,r3.T)
  print(K)

In [ ]:
l=EV('/content/c1.png','/content/z-dimension.csv',0.025,1)
print("Eigenvector corresponding to the minimum eigenvalue:", l)
p_matrix(l)

Eigenvector corresponding to the minimum eigenvalue: [-2.12938764e-12  1.39488299e-13  9.99942232e-01 -3.99976893e-04
 -6.26139765e-14 -1.80536358e-12 -2.89749959e-04  1.15899943e-07
 -7.32591517e-17  2.31375455e-16  1.07372384e-02 -4.29489537e-06]
[[ 4.94886201e-07 -1.52726487e-08  2.32821092e+05]
 [ 0.00000000e+00  4.20602302e-07  4.20603739e-07]
 [ 0.00000000e+00  0.00000000e+00  2.50000000e+03]]


In [ ]:
l=EV('/content/c2.png','/content/z-dimension.csv',25,2)
print("Eigenvector corresponding to the minimum eigenvalue:", l)
p_matrix(l)

Eigenvector corresponding to the minimum eigenvalue: [ 1.58427277e-13 -2.60505630e-14  9.97892042e-01 -4.98946021e-04
  1.00980422e-13  1.11043987e-14 -3.93408529e-03  1.96704264e-06
  1.78691395e-16  1.37762442e-16  6.47745795e-02 -3.23872897e-05]
[[1.39340905e-09 4.77432501e-09 3.08112241e+04]
 [0.00000000e+00 3.13705911e-09 3.13669777e-09]
 [0.00000000e+00 0.00000000e+00 2.00000000e+03]]


In [ ]:
l=EV('/content/c3.png','/content/z-dimension.csv',25,3)
print("Eigenvector corresponding to the minimum eigenvalue:", l)
p_matrix(l)

Eigenvector corresponding to the minimum eigenvalue: [-1.02407008e-13 -1.52240699e-14 -9.92094317e-01  5.95256590e-04
  1.07077975e-14 -1.59912448e-13  8.82177372e-04 -5.29306428e-07
  9.22878624e-17 -1.28517185e-16  1.25489952e-01 -7.52939711e-05]
[[ 1.36187137e-09  1.10879287e-10 -1.31762783e+04]
 [ 0.00000000e+00  2.12858478e-09  2.12859733e-09]
 [ 0.00000000e+00  0.00000000e+00  1.66666667e+03]]


In [ ]:
l=EV('/content/c4.png','/content/z-dimension.csv',25,4)
print("Eigenvector corresponding to the minimum eigenvalue:", l)
p_matrix(l)


Eigenvector corresponding to the minimum eigenvalue: [-1.68984672e-13 -1.11657648e-14  9.77675230e-01 -6.84372661e-04
  7.98250355e-14 -2.87347403e-13 -5.23603686e-03  3.66522576e-06
  1.07306304e-16 -1.85183409e-16 -2.10055321e-01  1.47038724e-04]
[[ 1.12596151e-09 -2.34441178e-10 -6.64910032e+03]
 [ 0.00000000e+00  2.02819963e-09  2.02823475e-09]
 [ 0.00000000e+00  0.00000000e+00  1.42857143e+03]]


In [ ]:
l=EV('/content/c5.png','/content/z-dimension.csv',25,5)
print("Eigenvector corresponding to the minimum eigenvalue:", l)
p_matrix(l)

Eigenvector corresponding to the minimum eigenvalue: [-1.39517649e-13 -3.47547173e-14 -9.99032603e-01  7.99226082e-04
 -6.13896954e-14  3.65792466e-14  4.21171002e-03 -3.36936802e-06
 -1.21990770e-16  5.76076886e-17  4.37662036e-02 -3.50129629e-05]
[[ 2.90222985e-09  2.91498627e-09 -2.85332208e+04]
 [ 0.00000000e+00  2.04063014e-09  2.04099922e-09]
 [ 0.00000000e+00  0.00000000e+00  1.25000000e+03]]


In [ ]:
l=EV('/content/c6.png','/content/z-dimension.csv',25,6)
print("Eigenvector corresponding to the minimum eigenvalue:", l)
p_matrix(l)

Eigenvector corresponding to the minimum eigenvalue: [-5.38780755e-13  3.23412119e-14 -9.99256783e-01  8.99331104e-04
 -9.43104102e-14 -3.05574576e-13 -9.31686547e-04  8.38517824e-07
 -2.71249949e-16  3.68768538e-18  3.85253749e-02 -3.46728374e-05]
[[ 1.53177166e-08  3.69158163e-09 -2.88195849e+04]
 [ 0.00000000e+00  9.22333197e-09  9.22327863e-09]
 [ 0.00000000e+00  0.00000000e+00  1.11111111e+03]]


In [ ]:
l=EV('/content/c7.png','/content/z-dimension.csv',25,7)
print("Eigenvector corresponding to the minimum eigenvalue:", l)
p_matrix(l)

Eigenvector corresponding to the minimum eigenvalue: [ 9.68937452e-14 -6.05931648e-14  9.99916227e-01 -9.99916227e-04
  7.33527822e-14  4.30809902e-14  3.53445676e-03 -3.53445676e-06
  3.99925696e-17  5.55867049e-17 -1.24115387e-02  1.24115387e-05]
[[ 7.99434291e-09  4.25847374e-09 -8.05634376e+04]
 [ 0.00000000e+00  6.85539971e-09  6.85396253e-09]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+03]]


In [ ]:
l=EV('/content/c8.png','/content/z-dimension.csv',25,8)
print("Eigenvector corresponding to the minimum eigenvalue:", l)
p_matrix(l)

Eigenvector corresponding to the minimum eigenvalue: [-1.12417410e-13  3.00137290e-15  9.70448973e-01 -1.06749387e-03
 -7.66661040e-15 -1.55868374e-13  8.66546847e-04 -9.53201579e-07
 -1.62999722e-16  1.27262655e-16 -2.41302363e-01  2.65432599e-04]
[[ 4.26134606e-10  9.51437079e-12 -3.65610319e+03]
 [ 0.00000000e+00  5.87932226e-10  5.87933837e-10]
 [ 0.00000000e+00  0.00000000e+00  9.09090909e+02]]


In [ ]:
l=EV('/content/c9.png','/content/z-dimension.csv',25,9)
print("Eigenvector corresponding to the minimum eigenvalue:", l)
p_matrix(l)

Eigenvector corresponding to the minimum eigenvalue: [ 6.09650891e-14 -2.37877677e-16 -1.45617665e-01  1.74741198e-04
  5.77250921e-15  5.16942175e-14 -4.48362941e-04  5.38035543e-07
  4.15045936e-18  1.01135728e-17 -9.89340111e-01  1.18720813e-03]
[[5.10563092e-11 5.49970383e-12 1.22655549e+02]
 [0.00000000e+00 4.38133050e-11 4.38133090e-11]
 [0.00000000e+00 0.00000000e+00 8.33333333e+02]]
